# Emperor Penguin Population Estimation

### Mark B. & Katharine P. 
### GGS416: Satellite Image Analysis 

    This notebook is used as a guide on how to collect Sentinel-2 imagery and extracted colonies. Input data was replaced with filler for the user to change. Comments above lines indicate the "what" and "why" of code. Comments below a line of code represent the index at which the data should be formated.
    
The code is seperated into four sections:
        
#### 1. Image Collection
    Before images are used for analysis, data needs to be formatted into geojson in order for the Sentinel API to package the correct information. Preliminary observations on Sentinel Playground are going to be needed before collection. Antarctica is a dynamic ecosystem that produces satellite imagery containing more clouds than snow. Finding date ranges that hold limited cloud cover is vital for collection. 
    
#### 2. Image Correction
    Images recieved from the Sentinel API are almost always oversaturated. Correcting the pixel values by a factor of 10,000 creates observable images. Numpy is introduced in this section and lays the tracks of its subsequential use throughout the code. Additionally, the contributions of rasterio are displayed in this section.
    
#### 3. Image Clipping
    Clipping the images requires two variables, an image and a bounding box. Our corrected images are saved as GEOTIFFS, meaning the coordinate refernce system (CRS) is implemented throughout each value in an array. The bounding box must conincide with the shape, values, and CRS of the GEOTIFF.
    
#### 4. Image Extraction
    Guano can be extracted based on the pixel values of the array. Unfortunately, Python is incapable of seamless visualization. Through visual applications (ENVI, ArcGIS Pro), preliminary observations were made to determine the values of guano. A new image is made in which pixels are represented as either 0 or 1. 0 indictes no guano while 1 indicates likely areas of guano. A stipulation was created to determine if pixels were a 1 or a 0. Summing the pixels then allows for population estimation. This, however, is not in the code since there are multiple ways this can be done.

## Image Collection

In [ ]:
# These are all the packages needed for conducting this code.
# Additionally, a Copernicus account is required to pull images from the API. 
import folium
import geopandas as gpd
from geojson import Point, Feature, FeatureCollection, dump
from shapely.geometry import Point, Polygon
import pandas as pd
import sentinelsat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from matplotlib import pyplot as plt
import numpy as np
from rasterio.plot import show
from rasterio.features import shapes
from numpy import count_nonzero
import os
from rasterio.mask import mask
api = SentinelAPI('user', 'password','https://scihub.copernicus.eu/dhus') 
# 'user' and 'password' indicate your Copernicus username and password.

In [ ]:
# Data is placed into a list so inputs are straightforward.
data = [LONG,LAT,'DATEstart','DATEend','COLONYNAME'] 
# LONG, LAT are the coordinates of the site (this does not need to be in meters). 
# DATEstart and DATEend collectinvely state the date range of images.
# COLONYNAME is the name you wish to give the image.

# Function used for collection
def collecting(x):
    
    # Create bounding box so the entire colony is within the pulled image.
    # The values used to add and subtract are arbitrary. They are simply used to create a rough estimation of the study area.
        xmin= x[0] - 0.014438
        ymin= x[1] - 0.014438
        xmax= x[0] + 0.014438
        ymax= x[1] + 0.014438
        features = []
        polygon = Polygon([[xmin,
                  ymin],
                  [xmin,
                  ymax],
                  [xmax,
                  ymax],
                  [xmin,
                  ymax],
                  [xmin,
                  ymin]])
    
    # Appended features into an empty list that was created above. 
        features.append(Feature(geometry=polygon, properties={"Area": f"{x[4]}"}))
        feature_collection = FeatureCollection(features)
        
    # Write the GEOJSON of colony from the now filled list.
    # The information needs to be formatted into a GEOJSON so the API can understand what needs to be pulled.
        with open('{}.geojson'.format(x[4]), 'w') as f:
           dump(feature_collection, f)
        
    # Import GEOJSON 
        boundsdata = r'C:{}.geojson'.format(x[4])
        footprint = geojson_to_wkt(read_geojson(boundsdata))
        
    # Place GEOJSON into Sentinel's api and with the criteria of the product.
        products = api.query(footprint,
                     date = (x[2],x[3]),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0, 20),
                     limit=5)
    # Footprint indicates the area of interest (set before).
    # Date describes the date range for images to be pulled.
    # Platformname determines the satellite from which images will be pulled.
    # Processinglevel sets the amout of post processing an image has experienced. More info can be found: https://earthdata.nasa.gov/collaborate/open-data-services-and-software/data-information-policy/data-levels . 
    # Cloudcoverpercentage describes the range of acceptable cloud cover an image may hold.
    # Limit sets the maximum amount of images that can be pulled.
    
        api.download_all(products)
# The file will be downloaded into your directory.
collecting(data)

## Image Correction

In [ ]:
# Call bands 8,3, and 2.
# These are the bands needed to distinguish guano and snow.
image_filenameB02 = "XXXXB02_10m.jp2"
image_filenameB03 = "XXXXB03_10m.jp2"
image_filenameB08 = "XXXXB08_10m.jp2"
# XXX will be replaced with the name of the entire .jp2 image. XXX and 10m is consistent through all 3 images with the B0x changing with each.
# Example filepath: S2B_MSIL2A_20181223T174429_N0211_R069_T04CEV_20181223T212653.SAFE\GRANULE\L2A_T04CEV_A009390_20181223T174446\IMG_DATA\R10m

# Rasterio allows the user to read raster-type files.
my_imageB02 = rasterio.open(image_filenameB02)
my_imageB03 = rasterio.open(image_filenameB03)
my_imageB08 = rasterio.open(image_filenameB08)

In [ ]:
# Each respective band pulls its arrays.
blue = my_imageB02.read(1)
green = my_imageB03.read(1)
nir = my_imageB08.read(1)

# Scale values for display purposes.
# The raw images are incapable of visually representing the study areas. When corrected by a factor of 10,000, the images become legiable.
def scale(band): 
    return band /10000

# Run each band through the above function.
blue = scale(my_imageB02.read(1))
green = scale(my_imageB03.read(1))
nir = scale(my_imageB08.read(1))

In [ ]:
# The numpy package is used primarily for its mechanics use in multidimensional arrays.
# Stack bands in desired order
x = np.dstack((nir, green, blue))

# pypplot allows us to observe our image on the go.
show(x)

In [ ]:
# Again, numpy's ability to alter arrays is showcased.
# Reorder axis for writing
guano_reordered = np.moveaxis(x, [0, 1, 2], [1, 2, 0])
guano_reordered.shape

In [ ]:
# Just asrasterio can read an image, here, writing an image is displayed
# Write .tif of the 8,3, and 2 combination
with rasterio.open(
    'COLONYNAME.tif',                           
    'w',                                            
    driver='GTiff',                       #specification of export driver type          
    height=guano_reordered.shape[1],      #specification of height             
    width=guano_reordered.shape[2],       #specification of width            
    count=guano_reordered.shape[0],       #specification of bands           
    dtype=guano_reordered.dtype,          #specification of datatype           
    crs=my_imageB02.profile['crs'],       #specification of coordinate reference system             
    transform=my_imageB02.profile['transform']   #specification of transformation      
    ) as my_raster_writer:
        my_raster_writer.write(guano_reordered)    
# The file will write itself into your directory

print('Finished writing guano_reordered')

## Image Clipping

In [ ]:
# Import rasterio and file.
import rasterio

# image_file must be reset to the corrected image.
image_file = "COLONYNAME.tif"
# COLONYNAME indicates the corrected image.

# Again, no processing can be conducted if the image is not read using rasterio.
my_image = rasterio.open(image_file)
my_image

In [ ]:
# This is useful for clarifying the indexing and bounds of the image.
# For example: If your xmin (in the following block) is not greater than xminb then your study area then your study area is not completely within the image.
xminb, yminb, xmaxb, ymaxb = my_image.bounds

xminb, yminb, xmaxb, ymaxb

In [ ]:
# Using rasterio (more specifically, its mask function), clipping will be conducted.
with rasterio.open(image_file) as img:
    clipped, transform = mask(img, my_geojson, crop=True)

# Determine the coordinates of the desired study area.
# This should be known before running the code.
# The easiest way is to put it into a visual application (i.e. ArcGIS, QGIS, ENVI)
# Ensure that the input coordinates are in meters since those are the units of the projected coordinate system.
xmin = XXXX
ymin = XXXX
xmax = XXXX
ymax = XXXX

# Again, create the GEOJSON of the footprint 
my_geojson = [
    {
        "type": "Polygon",
        "coordinates": [  
          [
            [
              xmin,
              ymin
            ],
            [
              xmax,
              ymin
            ],
            [
              xmax,
              ymax
            ],
            [
              xmin,
              ymax
            ],
            [
              xmin,
              ymin
            ]
          ],
        ]
      }
    ]

# Above, we set the meta data of transform and clipped to equal each other. The combination of the two contribute to the clipped image's meta data.
meta = my_image.meta.copy()
meta.update(
    {
    
        "transform": transform,
        "height":clipped.shape[1],
        "width":clipped.shape[2]
    }
)

# Write out the new .tif
with rasterio.open('CLIPPEDCOLONYNAME.tif', 'w', **meta) as dst:
    dst.write(clipped)

## Image Extraction

In [ ]:
# Use rasterio to open the clipped image
my_image = rasterio.open('images/clipped/clippedXXX2021.tif')
# A great way to organize the data is to put the clipped images into their own file

# Display image on the go
show(my_image.read())

In [1]:
# Since we stacked the three bands into a numpy array earlier, we can use rasterio to parse out the data again.
nir = my_image.read(1)
green = my_image.read(2)
blue = my_image.read(3)

# For us to be able analyze the pixels, the datatype needs to be specified.
# Since the data needs to be specified in each array, we use numpy and assign float32.
nir = np.float32(nir)
green = np.float32(green)
blue = np.float32(blue)

NameError: name 'my_image' is not defined

In [ ]:
# A matching array of the study area needs to be created.
guano_index = np.zeros(blue.shape)

# The empty array is then given a stipulation that needs to be satisfied before a value of 1 is assigned to a pixel.
# Values are based on the three read images in the above cell.
guano_index[(nir > green) &  (green > blue) & (blue < 1)] = 1
# This stipulation was determined by observations made in ENVI. 
# More often then not, pixels that clearly indicated areas of guano had the highest reflectance in the NIR band (8)
# It would then decrease linearly when moving towards the green (3) and blue (2) band.
# An additional statement was needed to differentiate shadows and guano.
# Shadows and guano had the most variance within the 8 and 3 bands. 2 would then be used to seperate these features.
# Guano was almost always below a value of 1 within band 2.

plt.imshow(guano_index)
plt.colorbar(shrink=.7)

# count_nonzero counts the total values that do not equal 0.
# This then gives us the total pixels containing guano.
count_nonzero(guano_index, axis = None)

In [ ]:
# Reiterating the process in the correction, numpy was used to reorder the axis of the array.
x = np.dstack((guano_index)) 
guano_reordered = np.moveaxis(x, [0, 1, 2], [1, 2, 0])
guano_reordered.shape
guano_rereordered = np.moveaxis(guano_reordered,[0,1,2],[1,0,2])
guano_rereordered.shape

In [ ]:
# Finally, export the file as a tif
with rasterio.open(
    'NAME.tif',    #NAME is yours to decide                       
    'w',                                            
    driver='GTiff',                                     #specification of export driver type
    height=guano_rereordered.shape[1],                  #specification of height
    width=guano_rereordered.shape[2],                   #specification of width
    count=guano_rereordered.shape[0],                   #specification of bands
    dtype=guano_rereordered.dtype,                      #specification of datatype
    crs=my_image.profile['crs'],                    #specification of coordinate reference system
    transform=my_image.profile['transform']         #specification of transformation
    ) as my_raster_writer:
        my_raster_writer.write(guano_rereordered)       

print('Finished writing reordered guano')